In [95]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import itertools

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB


from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix ,classification_report, accuracy_score
from sklearn.metrics import roc_auc_score,roc_curve,scorer
from sklearn.metrics import f1_score

from sklearn.impute import SimpleImputer


from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn import tree

from xgboost import XGBClassifier

from lightgbm import LGBMClassifier

import pandas_profiling

from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from imblearn.over_sampling import RandomOverSampler

import plotly.plotly as py
import plotly.figure_factory as ff
import plotly
import plotly.graph_objs as go
import plotly.tools as tls


In [96]:
df_churn = pd.read_csv('Telco-Customer-Churn.csv')

In [143]:
df_churn;

#### Dropping "CustomerID" as it provides no signal

In [98]:
df_churn.drop(['customerID'], axis=1, inplace=True)

#### Replacing empty spaces with null values in "TotalCharges"

In [99]:
df_churn['TotalCharges'] = df_churn["TotalCharges"].replace(" " , np.nan)

#### Dropping nulls from "TotalCharges" 

In [100]:
df_churn = df_churn[df_churn["TotalCharges"].notnull()]
df_churn = df_churn.reset_index()[df_churn.columns]

#### Converting strings to floats in "TotalCharges"

In [101]:
df_churn["TotalCharges"] = df_churn["TotalCharges"].astype(float)

#### replacing 'No internet service' to 'No' for columns which show 'No internet service'

Making it more consistent, logically sound

In [102]:
replace_columns = ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport','StreamingTV', 'StreamingMovies']

for i in replace_columns : 
    df_churn[i]  = df_churn[i].replace({'No internet service' : 'No'})

In [103]:
'''

''';
    
# #replace values
# df_churn["SeniorCitizen"] = df_churn["SeniorCitizen"].replace({1:"Yes",0:"No"})


### Function to bin tenures

I might come back to this and rebin later. Especially curious about more bins to differntiate users with less than 2 years further.

In [104]:
def bin_tenure(df_churn):
    
    if df_churn["tenure"] <= 12 :
        return "Tenure_0_to_12"
    elif (df_churn["tenure"] > 12) & (df_churn["tenure"] <= 24 ):
        return "Tenure_12_to_24"
    elif (df_churn["tenure"] > 24) & (df_churn["tenure"] <= 48) :
        return "Tenure_24_to_48"
    elif (df_churn["tenure"] > 48) & (df_churn["tenure"] <= 60) :
        return "Tenure_48_to_60"
    elif df_churn["tenure"] > 60 :
        return "Tenure_60+"

### binning tenure 

In [105]:
df_churn["tenure_bin"] = df_churn.apply(lambda df_churn:bin_tenure(df_churn), axis = 1)

In [106]:
# cat_cols   = df_churn.nunique()[df_churn.nunique() <= 5].keys().tolist()
# cat_cols;

In [107]:
# Separating churn and non-churn
df_only_churn     = df_churn[df_churn["Churn"] == "Yes"]
df_not_churn = df_churn[df_churn["Churn"] == "No"]

# Separating columns by numerical and categorical, target column is "Churn"
target_col = ["Churn"]

cat_cols   = df_churn.nunique()
cat_cols   = df_churn.nunique()[df_churn.nunique() <= 5].keys().tolist()
cat_cols   = [col for col in cat_cols if col not in target_col]
num_cols   = [col for col in df_churn.columns if col not in cat_cols + target_col]


# get binary columns with 2 values
bin_cols   = df_churn.nunique()[df_churn.nunique() == 2].keys().tolist()
# get columns with more than 2 values
multi_cols = [i for i in cat_cols if i not in bin_cols]

In [108]:
df_churn;

### Standardizing
Standardizing the columns with numerical data, i.e. 'tenure', 'MonthlyCharges', 'TotalCharges'

In [109]:
std = StandardScaler()
df_scaled = std.fit_transform(df_churn[num_cols])
df_scaled = pd.DataFrame(df_scaled,columns=num_cols)

In [110]:
df_scaled;

backup df as pickle

In [111]:

# df_scaled.to_pickle("./df_churn.pkl")


import from pickle

### Merging the standardized columns into original df, dropping old values

In [112]:
df_churn_scaled = df_churn.drop(columns = num_cols,axis = 1)
df_churn_scaled = df_churn_scaled.merge(df_scaled,left_index=True,right_index=True,how = "left")

In [113]:
df_churn_scaled;

### One Hot Encoding

In [114]:
df_churn_scaled = pd.get_dummies(data = df_churn_scaled,columns = multi_cols, drop_first = True )
df_churn_scaled = pd.get_dummies(data = df_churn_scaled,columns = bin_cols, drop_first = True )

Pickle df_churn_scaled as backup

In [115]:
df_churn_scaled;

In [116]:

# df_churn_scaled.to_pickle("./df_churn_scaled_with_1_hot_encoding.pkl")


In [117]:
# df_churn_scaled = pd.read_pickle("./df_churn_scaled_with_1_hot_encoding.pkl")

## Model

### Train Test Validation Split

In [118]:
train_val , test = train_test_split(df_churn_scaled,test_size = .20 ,random_state = 1988)
train , val = train_test_split(train_val, test_size = .20 ,random_state = 2014)

In [130]:
cols    = [i for i in df_churn_scaled.columns if i not in target_col]

target_col = ["Churn_Yes"]

train_X = train[cols]
train_Y = train[target_col]
val_X = val[cols]
val_Y = val[target_col]
test_X  = test[cols]
test_Y  = test[target_col]


#### Function to show ROC

In [131]:
def plot_roc(test_y, preds, probs):
    '''
    plots ROC and prints out AUC
    '''
    model_roc_auc = roc_auc_score(test_y, preds) 
    print ("Area under curve : ",model_roc_auc,"\n")

    fpr, tpr, thresholds = roc_curve(test_y, probs[:,1])

    plt.plot(fpr, tpr, lw = 2)
    plt.plot([0,1],[0,1], c='violet', ls='--')
    plt.xlim([-0.05,1.05])
    plt.ylim([-0.05,1.05])


    plt.xlabel('False positive rate')
    plt.ylabel('True positive rate')
    plt.title('ROC curve');

#### Function to compare baseline models

In [132]:
def get_baseline_models_AUCs(train_x = train_X, test_x = val_X,
                            train_y = train_Y, test_y = val_Y):
    '''
    This function prints out the AUCs for all baseline models.
    It returns a dict with all the models as keys, and the AUCs as values.
    '''
    result = {}
    models = {"Logistic Regression" : LogisticRegression(), 
              "KNN" : KNeighborsClassifier(n_neighbors = 6),
              "Support Vector Classifier" : SVC(probability=True),
              "Random Forest": RandomForestClassifier(random_state= 31),
              "Gaussian Naive Bayes" : GaussianNB(),
#               "Multinomial Naive Bayes" : MultinomialNB(),
              "Light GBM Classifier" : LGBMClassifier(),
              "XGBoost Classifier" : XGBClassifier()
             }
    
    for key, algorithm in models.items():
        algorithm.fit(train_x, train_y)
        predictions   = algorithm.predict(test_x)
        probabilities = algorithm.predict_proba(test_x)
        model_roc_auc = roc_auc_score(test_y, predictions) 
        print (f'{key} :  {model_roc_auc}\n')
        result[key] = model_roc_auc
    
    return result

In [133]:
get_baseline_models_AUCs()

Logistic Regression :  0.701724124594881

KNN :  0.6520269486320707

Support Vector Classifier :  0.6595221336138549

Random Forest :  0.6411805883217438

Gaussian Naive Bayes :  0.7630121516363327

Light GBM Classifier :  0.688048699364959

XGBoost Classifier :  0.6961433444459055



{'Logistic Regression': 0.701724124594881,
 'KNN': 0.6520269486320707,
 'Support Vector Classifier': 0.6595221336138549,
 'Random Forest': 0.6411805883217438,
 'Gaussian Naive Bayes': 0.7630121516363327,
 'Light GBM Classifier': 0.688048699364959,
 'XGBoost Classifier': 0.6961433444459055}

## Model Pipeline

In [148]:
def make_model_prediction_coefs(cols, cf, algorithm = LogisticRegression(), 
                                train_x = train_X , test_x = val_X ,
                                train_y = train_Y ,test_y = val_Y ) :
    
    '''
    This function fits the model, prints a summary including AUC curve, accuracy score, F1 score
    It also produces 3 plotly graphs: A confusion matrix, an ROC curve, and a bar graph showing
    feature importance for each feature.
    
    INPUT:
    It takes columns, algorithm, train_df for x and y, test_df for x and y bas inputs
    
    cf is an input string that is either "coefficients" or "features"
    
    DEFAULT:
    Its default algorithm is LogisticRegression()
    default train dfs are train_X and train_Y
    default test dfs are val_X and val_Y
    '''
    
    
    algorithm.fit(train_x, train_y)
    
    predictions   = algorithm.predict(test_x)  
    probabilities = algorithm.predict_proba(test_x)

    fpr, tpr, thresholds = roc_curve(test_y, probabilities[:,1])
    
    if   cf == "coefficients":
        coefficients  = pd.DataFrame(algorithm.coef_.ravel())
    elif cf == "features":
        coefficients  = pd.DataFrame(algorithm.feature_importances_)
        
    column_df       = pd.DataFrame(cols)
    coef_summary    = (pd.merge(coefficients,column_df,left_index= True,
                              right_index= True, how = "left"))
    coef_summary.columns = ["coefficients","features"]
    coef_summary    = coef_summary.sort_values(by = "coefficients",ascending = False)
    
    print("_____________________________________________________________")   
    print("\n Results : \n", classification_report(test_y,predictions))
    print("Accuracy - score : ", accuracy_score(test_y,predictions))
    
    
    conf_matrix = confusion_matrix(test_y,predictions)
    
    

    # making roc curve, redundant
#     plot_roc(test_y, preds = predictions, probs = probabilities)
    
    # this can be commented out if using non-plotly roc curve above: plot_roc()
    model_roc_auc = roc_auc_score(test_y, predictions) 
    print ("Area under curve : ",model_roc_auc,"\n")
    
    fig = tls.make_subplots(rows=2, cols=2, specs=[[{}, {}], [{'colspan': 2}, None]],
                            subplot_titles=('Confusion Matrix',
                                            'Receiver operating characteristic',
                                            'Feature Importances'),
                            print_grid=False)
    
    
    
    bars = go.Bar(x = coef_summary["features"], y = coef_summary["coefficients"],
                    name = "coefficients",
                    marker = dict(color = coef_summary["coefficients"],
                                  colorscale = "Jet",
                                  line = dict(width = .6, color = "black")))
    
    roc_line = go.Scatter(x = fpr, y = tpr,
                        name = "Roc : " + str(model_roc_auc),
                        line = dict(color = ('rgb(25, 96, 167)'),width = 2))
    
    roc_stand = go.Scatter(x = [0,1],y=[0,1],
                        line = dict(color = ('rgb(215, 12, 24)'),width = 2,
                        dash = 'dot'))
    
    c_matrix = go.Heatmap(z = conf_matrix ,
                        x = ["Non churn","Churn"],
                        y = ["Non churn","Churn"],
                        showscale  = False, colorscale = "Jet",
                        name = "matrix")
    
    
    
    
    fig.append_trace(bars,2,1)
    fig.append_trace(roc_line,1,2)
    fig.append_trace(c_matrix,1,1)
    fig.append_trace(roc_stand,1,2)
    
    fig['layout'].update(showlegend = False, title="Model performance" ,
                         autosize = False,height = 1000,width = 850,
                         plot_bgcolor = 'rgba(230,230,230, 0.95)',
                         paper_bgcolor = 'rgba(230,230,230, 0.95)',
                         margin = dict(b = 185))
    
    fig["layout"]["xaxis2"].update(dict(title = "false positive rate"))
    fig["layout"]["yaxis2"].update(dict(title = "true positive rate"))
    fig["layout"]["xaxis1"].update(dict(showgrid = True,tickfont = dict(size = 10),
                                        tickangle = 90))
#     print('\nConfusion Matrix:')
#     print(conf_matrix)
    
    
    return(algorithm, py.iplot(fig, filename = 'jupyter-table1'))


In [145]:
(logit_model, logit_graph) = make_model_prediction_coefs(cols = cols, cf = "coefficients")
logit_graph

_____________________________________________________________

 Results : 
               precision    recall  f1-score   support

           0       0.82      0.89      0.85       803
           1       0.65      0.52      0.57       322

   micro avg       0.78      0.78      0.78      1125
   macro avg       0.73      0.70      0.71      1125
weighted avg       0.77      0.78      0.77      1125

Accuracy - score :  0.7813333333333333
Area under curve :  0.701724124594881 



## KNN

In [ ]:
knn = KNeighborsClassifier(algorithm = 'auto', leaf_size=30,
           metric_params = None, n_jobs=-1, n_neighbors = 6, p = 2,
           weights='uniform')

In [ ]:
knn.fit(train_X,train_Y)

In [ ]:
predictions_knn   = knn.predict(val_X)
probabilities_knn = knn.predict_proba(val_X)

In [ ]:
print("\n Classification report: \n",classification_report(val_Y ,predictions_knn))
print("Accuracy Score: ",accuracy_score(val_Y ,predictions_knn))

In [ ]:
plot_roc(val_Y = val_Y, preds = predictions_knn, probs = probabilities_knn)

## Random Forest

In [ ]:
forest = RandomForestClassifier()

## SMOTE

In [137]:
train_X_SMOTE, train_Y_SMOTE = SMOTE(random_state = 1911).fit_sample(train_X, train_Y)
val_X_SMOTE, val_Y_SMOTE = SMOTE(random_state = 1911).fit_sample(val_X, val_Y)

In [138]:
(logit_model_smote, logit_graph_smote) = make_model_prediction_coefs(cols = cols, cf = "coefficients", 
                                                                     train_x =train_X_SMOTE, train_y = train_Y_SMOTE, 
                                                                     test_x = val_X_SMOTE, test_y = val_Y_SMOTE)
logit_graph_smote

_____________________________________________________________

 Results : 
               precision    recall  f1-score   support

           0       0.79      0.71      0.75       803
           1       0.74      0.81      0.77       803

   micro avg       0.76      0.76      0.76      1606
   macro avg       0.76      0.76      0.76      1606
weighted avg       0.76      0.76      0.76      1606

Accuracy - score :  0.7615193026151931
Area under curve :  0.7615193026151931 



In [142]:
get_baseline_models_AUCs(train_x = train_X_SMOTE, test_x = val_X_SMOTE,
                            train_y = train_Y_SMOTE, test_y = val_Y_SMOTE)

Logistic Regression :  0.7615193026151931

KNN :  0.7098381070983811

Support Vector Classifier :  0.7770859277708593

Random Forest :  0.7571606475716064

Gaussian Naive Bayes :  0.772104607721046

Light GBM Classifier :  0.8337484433374845

XGBoost Classifier :  0.8356164383561643



{'Logistic Regression': 0.7615193026151931,
 'KNN': 0.7098381070983811,
 'Support Vector Classifier': 0.7770859277708593,
 'Random Forest': 0.7571606475716064,
 'Gaussian Naive Bayes': 0.772104607721046,
 'Light GBM Classifier': 0.8337484433374845,
 'XGBoost Classifier': 0.8356164383561643}

SMOTE lead to an increased performance. 

## ADASYN

In [139]:
train_X_ADASYN, train_Y_ADASYN = ADASYN(random_state = 1990).fit_sample(train_X, train_Y)
val_X_ADASYN, val_Y_ADASYN = ADASYN(random_state = 1990).fit_sample(val_X, val_Y)

In [140]:
(logit_model_adasyn, logit_graph_adasyn) = make_model_prediction_coefs(cols = cols, cf = "coefficients", 
                                                                     train_x =train_X_ADASYN, train_y = train_Y_ADASYN, 
                                                                     test_x = val_X_ADASYN, test_y = val_Y_ADASYN)
logit_graph_adasyn

_____________________________________________________________

 Results : 
               precision    recall  f1-score   support

           0       0.78      0.63      0.70       803
           1       0.69      0.82      0.75       787

   micro avg       0.73      0.73      0.73      1590
   macro avg       0.74      0.73      0.72      1590
weighted avg       0.74      0.73      0.72      1590

Accuracy - score :  0.7257861635220125
Area under curve :  0.726758455031244 



In [141]:
get_baseline_models_AUCs(train_x = train_X_ADASYN, test_x = val_X_ADASYN,
                            train_y = train_Y_ADASYN, test_y = val_Y_ADASYN)

Logistic Regression :  0.726758455031244

KNN :  0.6978144220925026

Support Vector Classifier :  0.7441527246143353

Random Forest :  0.7609211644389449

Gaussian Naive Bayes :  0.7491490772373612

Light GBM Classifier :  0.8281768969920612

XGBoost Classifier :  0.836803220451895



{'Logistic Regression': 0.726758455031244,
 'KNN': 0.6978144220925026,
 'Support Vector Classifier': 0.7441527246143353,
 'Random Forest': 0.7609211644389449,
 'Gaussian Naive Bayes': 0.7491490772373612,
 'Light GBM Classifier': 0.8281768969920612,
 'XGBoost Classifier': 0.836803220451895}

ADASYN doesn't do as well was SMOTE.